In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
train_rice = pd.read_csv('./datalab/231753/train_rice.csv',encoding='gb2312')
train_weather = pd.read_csv('/home/tianchi/myspace/train_weather.csv')
testA = pd.read_csv('./datalab/231753/testA.csv',header=None,encoding='gb2312')
# 我们是testA，所以就预测早稻产量就行了

In [ ]:
# index_unknown=train_weather[[x in ['*','/'] for x in list(train_weather['日降水量（mm）'])]].index
# for index in index_unknown:
#     train_weather.loc[index,'日降水量（mm）']=train_weather[index-4:index+3]['日降水量（mm）'][train_weather['日降水量（mm）']!='*'][train_weather['日降水量（mm）']!='/'].astype(float).mean()
# # 日降水量中的符号替换为近7天的均值
train_rice.shape

In [ ]:
# index_unknown=train_weather[train_weather['日照时数（单位：h)']=='/'].index
# for index in index_unknown:
#     train_weather.loc[index,'日照时数（单位：h)']=train_weather[index-4:index+3]['日照时数（单位：h)'][train_weather['日照时数（单位：h)']!='/'].astype(float).mean()

# testA['2018年早稻']  = train_rice['2017年早稻']
# testA.to_csv('/home/tianchi/myspace/testA.csv')

In [ ]:
# # 将日平均风速替换成临近七天的均值
# index_unknown=train_weather[train_weather['日照时数（单位：h)']=='/'].index
# for index in index_unknown:
#     train_weather.loc[index,'日照时数（单位：h)']=train_weather[index-4:index+3]['日照时数（单位：h)'][train_weather['日照时数（单位：h)']!='/'].astype(float).mean()
train_weather.head()
# 每一个区县四年，每一年365天的天气，全都记录下来了

In [ ]:
# # 将日平均风速替换成临近七天的均值
# index_unknown=train_weather[train_weather['日照时数（单位：h)']=='/'].index
# for index in index_unknown:
#     train_weather.loc[index,'日照时数（单位：h)']=train_weather[index-4:index+3]['日照时数（单位：h)'][train_weather['日照时数（单位：h)']!='/'].astype(float).mean()
train_weather.head()
# 每一个区县四年，每一年365天的天气，全都记录下来了

In [ ]:
other_county = [x for x in train_weather['区县id'].unique() if x not in testA[0].unique()]
for drop_county in other_county:
    train_weather=train_weather[~train_weather['区县id'].isin([drop_county])]
# 有几个区县在最后要提交的测试集里面没有，先给删了


In [ ]:
# # 平均温度也换成近7天的均值
# index_unknown=train_weather[train_weather['日平均风速(单位：m/s)']=='/'].index
# for index in index_unknown:
#     train_weather.loc[index,'日平均风速(单位：m/s)']=train_weather[index-4:index+3]['日平均风速(单位：m/s)'][train_weather['日平均风速(单位：m/s)']!='/'].astype(float).mean()
len(train_weather['区县id'].unique())

In [ ]:
# # 湿度和气压也这么替换
# index_unknown=train_weather[train_weather['日平均温度（单位：℃）']=='/'].index
# for index in index_unknown:
#     train_weather.loc[index,'日平均温度（单位：℃）']=train_weather[index-4:index+3]['日平均温度（单位：℃）'][train_weather['日平均温度（单位：℃）']!='/'].astype(float).mean()
# index_unknown=train_weather[train_weather['日相对湿度（单位：%）']=='/'].index
# for index in index_unknown:
#     train_weather.loc[index,'日相对湿度（单位：%）']=train_weather[index-4:index+3]['日相对湿度（单位：%）'][train_weather['日相对湿度（单位：%）']!='/'].astype(float).mean()

In [ ]:
# train_weather[train_weather['日降水量（mm）']=='*'].index
# # 检查一下

In [ ]:
# train_weather[train_weather['日平均气压（单位：hPa）']=='/'].index
# # 气压这个也需要转换一下


In [ ]:


# index_unknown=train_weather[train_weather['日平均气压（单位：hPa）']=='/'].index
# for index in index_unknown:
#     train_weather.loc[index,'日平均气压（单位：hPa）']=train_weather[index-4:index+3]['日平均气压（单位：hPa）'][train_weather['日平均气压（单位：hPa）']!='/'].astype(float).mean()


In [ ]:
# train_weather.to_csv('/home/tianchi/myspace/train_weather.csv')
# 保存一下,不过在这种云平台就没什么用了，只能保存到个人空间里面

In [ ]:
train_weather.loc[:,['日照时数（单位：h)','日平均风速(单位：m/s)','日降水量（mm）','日最高温度（单位：℃）','日最低温度（单位：℃）','日平均温度（单位：℃）','日相对湿度（单位：%）','日平均气压（单位：hPa）']].astype(float).shape

In [ ]:
earlyrice=train_weather[[x in [3,4,5,6,7] for x in list(train_weather['月份'])]]
laterice=train_weather[[x in [6,7,8,9,10] for x in list(train_weather['月份'])]]
# 划分早晚稻

In [ ]:
# 每个地区每年每一周数据取均值作为训练数据
from itertools import product
X_earlyrice=pd.DataFrame(columns=['区县id','年份','日照时数（单位：h)','日平均风速(单位：m/s)','日降水量（mm）','日最高温度（单位：℃）',
                        '日最低温度（单位：℃）','日平均温度（单位：℃）','日相对湿度（单位：%）','日平均气压（单位：hPa）'])
X_laterice=pd.DataFrame(columns=['区县id','年份','日照时数（单位：h)','日平均风速(单位：m/s)','日降水量（mm）','日最高温度（单位：℃）',
                        '日最低温度（单位：℃）','日平均温度（单位：℃）','日相对湿度（单位：%）','日平均气压（单位：hPa）'])

for qxid,year in list(product(list(set(list(earlyrice['区县id']))),list(set(list(earlyrice['年份']))))):
    curr=earlyrice[earlyrice['区县id']==qxid][earlyrice['年份']==year]
    n=len(curr.index)//7*7
    if n!=147:
        n=147
        curr=curr[['月份','日期','日照时数（单位：h)','日平均风速(单位：m/s)','日降水量（mm）','日最高温度（单位：℃）','日最低温度（单位：℃）',
                   '日平均温度（单位：℃）','日相对湿度（单位：%）','日平均气压（单位：hPa）']].astype(float).groupby(['月份','日期']).mean()
    x_new=curr[:n][['日照时数（单位：h)','日平均风速(单位：m/s)','日降水量（mm）','日最高温度（单位：℃）','日最低温度（单位：℃）',
                '日平均温度（单位：℃）','日相对湿度（单位：%）','日平均气压（单位：hPa）']].astype(float).groupby(np.arange(len(curr.index)//7*7)//7, axis=0).mean()
    x_new['年份']=year
    x_new['区县id']=qxid
    X_earlyrice=X_earlyrice.append(x_new,ignore_index=True)
    
for qxid,year in list(product(list(set(list(laterice['区县id']))),list(set(list(laterice['年份']))))):
    curr=laterice[laterice['区县id']==qxid][laterice['年份']==year]
    n=len(curr.index)//7*7
    if n!=147:
        n=147
        curr=curr[['月份','日期','日照时数（单位：h)','日平均风速(单位：m/s)','日降水量（mm）','日最高温度（单位：℃）',
                   '日最低温度（单位：℃）','日平均温度（单位：℃）','日相对湿度（单位：%）','日平均气压（单位：hPa）']].astype(float).groupby(['月份','日期']).mean()
    x_new=curr[:n][['日照时数（单位：h)','日平均风速(单位：m/s)','日降水量（mm）','日最高温度（单位：℃）','日最低温度（单位：℃）',
                '日平均温度（单位：℃）','日相对湿度（单位：%）','日平均气压（单位：hPa）']].astype(float).groupby(np.arange(len(curr.index)//7*7)//7, axis=0).mean()
    x_new['年份']=year
    x_new['区县id']=qxid
    X_laterice=X_laterice.append(x_new,ignore_index=True)

In [ ]:
# 开始划分验证集，测试集，训练集
X_earlyrice_val=X_earlyrice[X_earlyrice['年份']==2018]
X_laterice_val=X_laterice[X_laterice['年份']==2018]

from sklearn.model_selection import train_test_split

#index_earlyrice_train,index_earlyrice_test=train_test_split(list(product(list(set(list(X_earlyrice['区县id']))),[2015,2016,2017])),random_state=42)
#index_laterice_train,index_laterice_test=train_test_split(list(product(list(set(list(X_laterice['区县id']))),[2015,2016,2017])),random_state=42)
index_earlyrice_train,index_earlyrice_test=train_test_split(list(product(list(set(list(train_rice['区县id']))),[2015,2016,2017])),random_state=42)
index_laterice_train,index_laterice_test=train_test_split(list(product(list(set(list(train_rice['区县id']))),[2015,2016,2017])),random_state=42)

X_earlyrice_train=pd.DataFrame(columns=['区县id','年份','日照时数（单位：h)','日平均风速(单位：m/s)','日降水量（mm）','日最高温度（单位：℃）',
                        '日最低温度（单位：℃）','日平均温度（单位：℃）','日相对湿度（单位：%）','日平均气压（单位：hPa）'])
X_laterice_train=pd.DataFrame(columns=['区县id','年份','日照时数（单位：h)','日平均风速(单位：m/s)','日降水量（mm）','日最高温度（单位：℃）',
                        '日最低温度（单位：℃）','日平均温度（单位：℃）','日相对湿度（单位：%）','日平均气压（单位：hPa）'])
X_earlyrice_test=pd.DataFrame(columns=['区县id','年份','日照时数（单位：h)','日平均风速(单位：m/s)','日降水量（mm）','日最高温度（单位：℃）',
                        '日最低温度（单位：℃）','日平均温度（单位：℃）','日相对湿度（单位：%）','日平均气压（单位：hPa）'])
X_laterice_test=pd.DataFrame(columns=['区县id','年份','日照时数（单位：h)','日平均风速(单位：m/s)','日降水量（mm）','日最高温度（单位：℃）',
                        '日最低温度（单位：℃）','日平均温度（单位：℃）','日相对湿度（单位：%）','日平均气压（单位：hPa）'])
y_earlyrice_train=[]
y_laterice_train=[]
y_earlyrice_test=[]
y_laterice_test=[]
for index in index_earlyrice_train:
    qxid,year=index[0],index[1]
    curr=X_earlyrice[X_earlyrice['区县id']==qxid][X_earlyrice['年份']==year]
    X_earlyrice_train=X_earlyrice_train.append(curr,ignore_index=True)
    y=float(train_rice[train_rice['区县id']==qxid][str(year)+'年早稻'].values)
    y_earlyrice_train.append(y)
for index in index_earlyrice_test:
    qxid,year=index[0],index[1]
    curr=X_earlyrice[X_earlyrice['区县id']==qxid][X_earlyrice['年份']==year]
    X_earlyrice_test=X_earlyrice_test.append(curr,ignore_index=True)
    y=float(train_rice[train_rice['区县id']==qxid][str(year)+'年早稻'].values)
    y_earlyrice_test.append(y)
for index in index_laterice_train:
    qxid,year=index[0],index[1]
    curr=X_laterice[X_laterice['区县id']==qxid][X_laterice['年份']==year]
    X_laterice_train=X_laterice_train.append(curr,ignore_index=True)
    y=float(train_rice[train_rice['区县id']==qxid][str(year)+'年早稻'].values)
    y_laterice_train.append(y)
for index in index_laterice_test:
    qxid,year=index[0],index[1]
    curr=X_laterice[X_laterice['区县id']==qxid][X_laterice['年份']==year]
    X_laterice_test=X_laterice_test.append(curr,ignore_index=True)
    y=float(train_rice[train_rice['区县id']==qxid][str(year)+'年早稻'].values)
    y_laterice_test.append(y)

In [ ]:
X_earlyrice_val.shape

In [ ]:
X_earlyrice_train.shape

In [ ]:
# 归一化处理
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

Xe_train=scaler.fit_transform(X_earlyrice_train[['日照时数（单位：h)','日平均风速(单位：m/s)','日降水量（mm）','日最高温度（单位：℃）',
                '日最低温度（单位：℃）','日平均温度（单位：℃）','日相对湿度（单位：%）','日平均气压（单位：hPa）']].values)
Xl_train=scaler.fit_transform(X_laterice_train[['日照时数（单位：h)','日平均风速(单位：m/s)','日降水量（mm）','日最高温度（单位：℃）',
                '日最低温度（单位：℃）','日平均温度（单位：℃）','日相对湿度（单位：%）','日平均气压（单位：hPa）']].values)
Xe_test=scaler.fit_transform(X_earlyrice_test[['日照时数（单位：h)','日平均风速(单位：m/s)','日降水量（mm）','日最高温度（单位：℃）',
                '日最低温度（单位：℃）','日平均温度（单位：℃）','日相对湿度（单位：%）','日平均气压（单位：hPa）']].values)
Xl_test=scaler.fit_transform(X_laterice_test[['日照时数（单位：h)','日平均风速(单位：m/s)','日降水量（mm）','日最高温度（单位：℃）',
                '日最低温度（单位：℃）','日平均温度（单位：℃）','日相对湿度（单位：%）','日平均气压（单位：hPa）']].values)
Xe_train=Xe_train.reshape(-1,21,8)
# 变成三维数组，然后和test长度相等
Xl_train=Xl_train.reshape(-1,21,8)
Xe_test=Xe_test.reshape(-1,21,8)
Xl_test=Xl_test.reshape(-1,21,8)
ye_train=np.array(y_earlyrice_train)
yl_train=np.array(y_laterice_train)
ye_test=np.array(y_earlyrice_test)
yl_test=np.array(y_laterice_test)

In [ ]:
Xe_val=scaler.fit_transform(X_earlyrice_val[['日照时数（单位：h)','日平均风速(单位：m/s)','日降水量（mm）','日最高温度（单位：℃）',
                '日最低温度（单位：℃）','日平均温度（单位：℃）','日相对湿度（单位：%）','日平均气压（单位：hPa）']].values)
Xe_val=Xe_val.reshape(-1,21,8)

In [ ]:
# LSTM训练
def make_batches(X,y,batch_size,n_steps):
    np.random.seed(2)
    batches=[]

    m=len(y)
    
    permutation = list(np.random.permutation(m)) 
    shuffled_X=X[permutation,:,:]
    shuffled_y=y[permutation]

    num_batch_size=m//batch_size
    for k in range(num_batch_size):
        batch_X=shuffled_X[k*batch_size:(k+1)*batch_size,:,:]
        batch_y=shuffled_y[k*batch_size:(k+1)*batch_size]
        batch=(batch_X,batch_y)
        batches.append(batch)
        
    if m%batch_size!=0:
        batch_X=shuffled_X[num_batch_size*batch_size:,:,:]
        batch_y=shuffled_y[num_batch_size*batch_size:]
        batch=(batch_X,batch_y)
        batches.append(batch)
        
    return batches

In [ ]:
tf.reset_default_graph()

n_steps=21
n_inputs=8
n_outputs=1
n_neurons=100

n_iterations = 2000
learning_rate=0.001
batch_size=8

X=tf.placeholder(tf.float32,[None,n_steps,n_inputs])
y=tf.placeholder(tf.float32,[None])

basic_cell=tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs,states=tf.nn.dynamic_rnn(basic_cell,X,dtype=tf.float32)

y_pre=tf.layers.dense(states,n_outputs)
loss=tf.reduce_mean(tf.square(y - y_pre), name = 'loss')
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op=optimizer.minimize(loss)

init=tf.global_variables_initializer()
saver=tf.train.Saver()

batches=make_batches(Xe_train,ye_train,batch_size,n_steps)

with tf.Session() as sess:
    init.run()
    for iterations in range(n_iterations):
        for batch in batches:
            X_batch,y_batch=batch[0],batch[1]
            sess.run(training_op,feed_dict={X:X_batch,y:y_batch})
        if iterations%100==0:
            test_loss=sess.run(loss,feed_dict={X:Xe_test,y:ye_test})
            train_loss=sess.run(loss,feed_dict={X:X_batch,y:y_batch})
            print("After %d training steps,Train loss is %g,Test loss is %g"%(iterations,train_loss,test_loss))
    saver.save(sess, "/home/tianchi/myspace/Rice_yield_prediction_LSTM_model")

In [ ]:
with tf.Session() as sess:
    saver.restore(sess,"/home/tianchi/myspace/Rice_yield_prediction_LSTM_model")
    ye_val_pre=sess.run(y_pre,feed_dict={X:Xe_val})
testA['2018年早稻']  = ye_val_pre
testA.to_csv('/home/tianchi/myspace/testA.csv')